<a href="https://colab.research.google.com/github/AslantheAslan/Node-classification-by-several-methods/blob/main/email_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
def text_to_array(path):

    my_file = open(path, "r")
    content = my_file.read()

    def split(word):
        return [char for char in word]

    content_list = content.split("\n")
    my_file.close()

    final = []
    for i in range(len(content_list)):
        final.append(split(content_list[i]))

    return np.asarray(final)

In [3]:
a = np.genfromtxt(r'/content/drive/My Drive/Colab Notebooks/Data/email_data/email-Eu-core.txt', delimiter=' ', dtype=int)
print(type(a))

<class 'numpy.ndarray'>


In [4]:
edges = list(map(tuple, a))
edges

[(0, 1),
 (2, 3),
 (2, 4),
 (5, 6),
 (5, 7),
 (8, 9),
 (10, 11),
 (12, 13),
 (12, 14),
 (15, 16),
 (17, 18),
 (12, 19),
 (20, 21),
 (20, 22),
 (23, 24),
 (23, 25),
 (23, 26),
 (23, 27),
 (23, 28),
 (23, 29),
 (23, 30),
 (23, 31),
 (23, 32),
 (23, 33),
 (23, 34),
 (23, 35),
 (23, 36),
 (23, 37),
 (23, 38),
 (23, 39),
 (23, 40),
 (41, 42),
 (43, 44),
 (45, 46),
 (47, 48),
 (49, 50),
 (41, 51),
 (52, 53),
 (54, 55),
 (54, 56),
 (54, 57),
 (54, 58),
 (54, 59),
 (60, 61),
 (54, 54),
 (62, 63),
 (64, 65),
 (62, 4),
 (66, 67),
 (68, 69),
 (42, 41),
 (70, 71),
 (72, 21),
 (71, 70),
 (73, 74),
 (75, 76),
 (75, 48),
 (77, 78),
 (77, 79),
 (80, 81),
 (80, 68),
 (19, 62),
 (82, 83),
 (22, 21),
 (82, 84),
 (21, 72),
 (41, 85),
 (41, 86),
 (41, 87),
 (82, 86),
 (88, 89),
 (90, 91),
 (92, 20),
 (41, 93),
 (41, 94),
 (41, 95),
 (89, 96),
 (89, 88),
 (97, 98),
 (97, 99),
 (97, 100),
 (97, 101),
 (97, 102),
 (103, 104),
 (51, 41),
 (82, 105),
 (90, 106),
 (62, 107),
 (108, 109),
 (108, 66),
 (108, 110),

In [5]:
b = np.genfromtxt(r'/content/drive/My Drive/Colab Notebooks/Data/email_data/email-Eu-core-department-labels.txt', delimiter=' ', dtype=int)
print(type(b))

<class 'numpy.ndarray'>


In [6]:
nodes_and_labels = list(map(tuple, b))

In [7]:
"""
I have just compiled this part to understand how tuples are indexed in an array
print(edges[2])
print(edges[2][0])
print(edges[2][1])
"""


'\nI have just compiled this part to understand how tuples are indexed in an array\nprint(edges[2])\nprint(edges[2][0])\nprint(edges[2][1])\n'

In [8]:
!pip install git+git://github.com/deepmind/jraph.git
!pip install flax
!pip install dm-haiku

  Cloning git://github.com/deepmind/jraph.git to /tmp/pip-req-build-t87309ip
  Running command git clone -q git://github.com/deepmind/jraph.git /tmp/pip-req-build-t87309ip


In [9]:
%matplotlib inline
import functools
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import jax.tree_util as tree
import jraph
import flax
import haiku as hk
import optax
import numpy as onp
import networkx as nx
from typing import Tuple

In [10]:
def get_email_dataset() -> jraph.GraphsTuple:
  """Returns GraphsTuple representing Zachary's karate club."""
  social_graph = list(map(tuple, a))
  # Add reverse edges.
  social_graph += [(edge[1], edge[0]) for edge in social_graph]
  n_club_members = 1005

  return jraph.GraphsTuple(
      n_node=jnp.asarray([n_club_members]),
      n_edge=jnp.asarray([len(social_graph)]),
      # One-hot encoding for nodes, i.e. argmax(nodes) = node index.
      nodes=jnp.eye(n_club_members),
      # No edge features.
      edges=None,
      globals=None,
      senders=jnp.asarray([edge[0] for edge in social_graph]),
      receivers=jnp.asarray([edge[1] for edge in social_graph]))

def get_ground_truth_assignments_for_email_dataset() -> jnp.ndarray:
  """Returns ground truth assignments for Zachary's karate club."""
  return jnp.asarray([element[1] for element in nodes_and_labels])

In [11]:
graph = get_email_dataset()

In [12]:
print(f'Number of nodes: {graph.n_node[0]}')
print(f'Number of edges: {graph.n_edge[0]}')

Number of nodes: 1005
Number of edges: 51142


In [13]:
def convert_jraph_to_networkx_graph(jraph_graph):
  nodes, edges, receivers, senders, _, _, _ = jraph_graph
  nx_graph = nx.DiGraph()
  if nodes is None:
    for n in range(jraph_graph.n_node[0]):
      nx_graph.add_node(n)
  else:
    for n in range(jraph_graph.n_node[0]):
      nx_graph.add_node(n, node_feature=nodes[n])
  if edges is None:
    for e in range(jraph_graph.n_edge[0]):
      nx_graph.add_edge(int(senders[e]), int(receivers[e]))
  else:
    for e in range(jraph_graph.n_edge[0]):
      nx_graph.add_edge(int(senders[e]), int(receivers[e]), edge_feature=edges[e])
  return nx_graph

def draw_jraph_graph_structure(jraph_graph: jraph.GraphsTuple):
  nx_graph = convert_jraph_to_networkx_graph(jraph_graph)
  pos = nx.spring_layout(nx_graph)
  nx.draw(nx_graph, pos=pos, with_labels = True, node_size=500, font_color='yellow')

In [ ]:
nx_graph = convert_jraph_to_networkx_graph(graph)
pos = nx.random_layout(nx_graph)
plt.figure(figsize=(12, 12))
nx.draw(nx_graph, pos=pos, with_labels = True, node_size=100, font_color='yellow')

In [15]:
"""
Here, the necessary functions to implement a GCN were defined
"""

class MLP(hk.Module):
  def __init__(self, features: jnp.ndarray):
    super().__init__()
    self.features = features

  def __call__(self, x: jnp.ndarray):
    layers = []
    for feat in self.features[:-1]:
      layers.append(hk.Linear(feat))
      layers.append(jax.nn.relu)
    layers.append(hk.Linear(self.features[-1]))

    mlp = hk.Sequential(layers)
    return mlp(x)

def apply_simplified_gcn(graph: jraph.GraphsTuple):
  # Unpack GraphsTuple
  nodes, _, receivers, senders, _, _, _ = graph

  # 1. Update node features
  # For simplicity, we will first use an identify function here, and replace it
  # with a trainable MLP block later.
  update_node_fn = lambda nodes: nodes
  nodes = update_node_fn(nodes)

  # 2. Aggregate node features over nodes in neighborhood
  # Equivalent to jnp.sum(n_node), but jittable
  total_num_nodes = tree.tree_leaves(nodes)[0].shape[0]
  aggregate_nodes_fn = jax.ops.segment_sum

  # Compute new node features by aggregating messages from neighboring nodes
  nodes = tree.tree_map(lambda x: aggregate_nodes_fn(x[senders], receivers,
                                        total_num_nodes), nodes)
  out_graph = graph._replace(nodes=nodes)
  return out_graph

def add_self_edges_fn(receivers, senders, total_num_nodes):
  """Adds self edges. Assumes self edges are not in the graph yet."""
  receivers = jnp.concatenate((receivers, jnp.arange(total_num_nodes)), axis=0)
  senders = jnp.concatenate((senders, jnp.arange(total_num_nodes)), axis=0)
  return receivers, senders

# Adapted from https://github.com/deepmind/jraph/blob/master/jraph/_src/models.py#L506
def GraphConvolution(
    update_node_fn,
    aggregate_nodes_fn=jax.ops.segment_sum,
    add_self_edges: bool = False,
    symmetric_normalization: bool = True):
  """Returns a method that applies a Graph Convolution layer.
  Graph Convolutional layer as in https://arxiv.org/abs/1609.02907,
  NOTE: This implementation does not add an activation after aggregation.
  If you are stacking layers, you may want to add an activation between
  each layer.
  Args:
    update_node_fn: function used to update the nodes. In the paper a single
      layer MLP is used.
    aggregate_nodes_fn: function used to aggregates the sender nodes.
    add_self_edges: whether to add self edges to nodes in the graph as in the
      paper definition of GCN. Defaults to False.
    symmetric_normalization: whether to use symmetric normalization. Defaults
      to True.
  Returns:
    A method that applies a Graph Convolution layer.
  """
  def _ApplyGCN(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
    """Applies a Graph Convolution layer."""
    nodes, _, receivers, senders, _, _, _ = graph

    # First pass nodes through the node updater.
    nodes = update_node_fn(nodes)
    # Equivalent to jnp.sum(n_node), but jittable
    total_num_nodes = tree.tree_leaves(nodes)[0].shape[0]
    if add_self_edges:
      # We add self edges to the senders and receivers so that each node
      # includes itself in aggregation.
      # In principle, a `GraphsTuple` should partition by n_edge, but in
      # this case it is not required since a GCN is agnostic to whether
      # the `GraphsTuple` is a batch of graphs or a single large graph.
      conv_receivers, conv_senders = add_self_edges_fn(receivers, senders, total_num_nodes)
    else:
      conv_senders = senders
      conv_receivers = receivers

    # pylint: disable=g-long-lambda
    if symmetric_normalization:
      # Calculate the normalization values.
      count_edges = lambda x: jax.ops.segment_sum(
          jnp.ones_like(conv_senders), x, total_num_nodes)
      sender_degree = count_edges(conv_senders)
      receiver_degree = count_edges(conv_receivers)

      # Pre normalize by sqrt sender degree.
      # Avoid dividing by 0 by taking maximum of (degree, 1).
      nodes = tree.tree_map(
          lambda x: x * jax.lax.rsqrt(jnp.maximum(sender_degree, 1.0))[:, None],
          nodes,
      )
      # Aggregate the pre-normalized nodes.
      nodes = tree.tree_map(
          lambda x: aggregate_nodes_fn(x[conv_senders], conv_receivers,
                                       total_num_nodes), nodes)
      # Post normalize by sqrt receiver degree.
      # Avoid dividing by 0 by taking maximum of (degree, 1).
      nodes = tree.tree_map(
          lambda x:
          (x * jax.lax.rsqrt(jnp.maximum(receiver_degree, 1.0))[:, None]),
          nodes,
      )
    else:
      nodes = tree.tree_map(
          lambda x: aggregate_nodes_fn(x[conv_senders], conv_receivers,
                                       total_num_nodes), nodes)
    # pylint: enable=g-long-lambda
    return graph._replace(nodes=nodes)

  return _ApplyGCN

def gcn(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
  """Defines a graph neural network with 3 GCN layers.
  Args:
    graph: GraphsTuple the network processes.

  Returns:
    output graph with updated node values.
  """
  gn = GraphConvolution(
      update_node_fn=lambda n: jax.nn.relu(hk.Linear(128)(n)),
      add_self_edges=True)
  graph = gn(graph)

  gn = GraphConvolution(
      update_node_fn=lambda n: jax.nn.relu(hk.Linear(256)(n)),
      add_self_edges=True)
  graph = gn(graph)

  gn = GraphConvolution(
      update_node_fn=hk.Linear(42))
  graph = gn(graph)
  return graph

def gcn_definition(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
  """Defines a GCN for the karate club task.
  Args:
    graph: GraphsTuple the network processes.

  Returns:
    output graph with updated node values.
  """
  gn = GraphConvolution(
      update_node_fn=lambda n: jax.nn.relu(hk.Linear(128)(n)),
      add_self_edges=True)
  graph = gn(graph)

  gn = GraphConvolution(
      update_node_fn=hk.Linear(42)) # output dim is 42 because we have 42 output classes.
  graph = gn(graph)
  return graph

In [16]:
def optimize_email_dataset(network, num_steps: int):
  """Solves the karate club problem by optimizing the assignments of students."""
  email_dataset = get_email_dataset()
  labels = get_ground_truth_assignments_for_email_dataset()
  params = network.init(jax.random.PRNGKey(42), email_dataset)

  @jax.jit
  def predict(params):
    decoded_graph = network.apply(params, email_dataset)
    return jnp.argmax(decoded_graph.nodes, axis=1)

  @jax.jit
  def prediction_loss(params):
    decoded_graph = network.apply(params, email_dataset)
    # We interpret the decoded nodes as a pair of logits for each node.
    log_prob = jax.nn.log_softmax(decoded_graph.nodes)
    # Here I have given some aprioric knowledge to the training set. I have
    # used 800 randomly shuffled samples to predict whole labels in the dataset
    sonuc = 0
    iterate = np.arange(1005)
    np.random.shuffle(iterate)
    for i in iterate[:800]:
      j = nodes_and_labels[i][1]
      sonuc += log_prob[i,j]
    return -(sonuc)

  opt_init, opt_update = optax.adam(1e-2)
  opt_state = opt_init(params)

  @jax.jit
  def update(params, opt_state):
    g = jax.grad(prediction_loss)(params)
    updates, opt_state = opt_update(g, opt_state)
    return optax.apply_updates(params, updates), opt_state

  @jax.jit
  def accuracy(params):
    decoded_graph = network.apply(params, email_dataset)
    return jnp.mean(jnp.argmax(decoded_graph.nodes, axis=1) == labels)

  for step in range(num_steps):
    print(f"step {step} accuracy {accuracy(params).item():.4f}")
    params, opt_state = update(params, opt_state)

  return predict(params)

In [17]:
network = hk.without_apply_rng(hk.transform(gcn_definition))
result = optimize_email_dataset(network, num_steps=500)
# After using 800 samples from the graph, accuracy appears to be %94.83 over 500 epochs.
# When you test the same network with 500 training samples, its throughput is observed as %87.5

step 0 accuracy 0.0189
step 1 accuracy 0.4159
step 2 accuracy 0.4776
step 3 accuracy 0.4876
step 4 accuracy 0.4776
step 5 accuracy 0.4567
step 6 accuracy 0.4348
step 7 accuracy 0.4199
step 8 accuracy 0.3940
step 9 accuracy 0.3701
step 10 accuracy 0.3214
step 11 accuracy 0.2886
step 12 accuracy 0.2726
step 13 accuracy 0.2736
step 14 accuracy 0.2876
step 15 accuracy 0.3065
step 16 accuracy 0.3254
step 17 accuracy 0.3672
step 18 accuracy 0.4129
step 19 accuracy 0.4726
step 20 accuracy 0.5184
step 21 accuracy 0.5493
step 22 accuracy 0.5791
step 23 accuracy 0.6090
step 24 accuracy 0.6299
step 25 accuracy 0.6338
step 26 accuracy 0.6348
step 27 accuracy 0.6378
step 28 accuracy 0.6428
step 29 accuracy 0.6448
step 30 accuracy 0.6517
step 31 accuracy 0.6597
step 32 accuracy 0.6667
step 33 accuracy 0.6687
step 34 accuracy 0.6796
step 35 accuracy 0.6965
step 36 accuracy 0.7055
step 37 accuracy 0.7104
step 38 accuracy 0.7164
step 39 accuracy 0.7224
step 40 accuracy 0.7294
step 41 accuracy 0.7333
st

In [18]:
######### GAT Implementation #########

# GAT implementation adapted from https://github.com/deepmind/jraph/blob/master/jraph/_src/models.py#L442.
def GAT(attention_query_fn,
        attention_logit_fn,
        node_update_fn=None,
        add_self_edges=True):
  """Returns a method that applies a Graph Attention Network layer.
  Graph Attention message passing as described in
  https://arxiv.org/pdf/1710.10903.pdf. This model expects node features as a
  jnp.array, may use edge features for computing attention weights, and
  ignore global features. It does not support nests.
  Args:
    attention_query_fn: function that generates attention queries
      from sender node features.
    attention_logit_fn: function that converts attention queries into logits for
      softmax attention.
    node_update_fn: function that updates the aggregated messages. If None,
      will apply leaky relu and concatenate (if using multi-head attention).
  Returns:
    A function that applies a Graph Attention layer.
  """
  # pylint: disable=g-long-lambda
  if node_update_fn is None:
    # By default, apply the leaky relu and then concatenate the heads on the
    # feature axis.
    node_update_fn = lambda x: jnp.reshape(
        jax.nn.leaky_relu(x), (x.shape[0], -1))

  def _ApplyGAT(graph):
    """Applies a Graph Attention layer."""
    nodes, edges, receivers, senders, _, _, _ = graph
    # Equivalent to the sum of n_node, but statically known.
    try:
      sum_n_node = nodes.shape[0]
    except IndexError:
      raise IndexError('GAT requires node features')

    # Pass nodes through the attention query function to transform
    # node features, e.g. with an MLP.
    nodes = attention_query_fn(nodes)

    total_num_nodes = tree.tree_leaves(nodes)[0].shape[0]
    if add_self_edges:
      # We add self edges to the senders and receivers so that each node
      # includes itself in aggregation.
      receivers, senders = add_self_edges_fn(receivers, senders, total_num_nodes)

    # We compute the softmax logits using a function that takes the
    # embedded sender and receiver attributes.
    sent_attributes = nodes[senders]
    received_attributes = nodes[receivers]
    att_softmax_logits = attention_logit_fn(
        sent_attributes, received_attributes, edges)

    # Compute the attention softmax weights on the entire tree.
    att_weights = jraph.segment_softmax(att_softmax_logits, segment_ids=receivers,
                                    num_segments=sum_n_node)

    # Apply attention weights.
    messages = sent_attributes * att_weights
    # Aggregate messages to nodes.
    nodes = jax.ops.segment_sum(messages, receivers, num_segments=sum_n_node)

    # Apply an update function to the aggregated messages.
    nodes = node_update_fn(nodes)

    return graph._replace(nodes=nodes)
  # pylint: enable=g-long-lambda
  return _ApplyGAT

In [19]:
def attention_logit_fn(sender_attr, receiver_attr, edges):
  del edges
  x = jnp.concatenate((sender_attr, receiver_attr), axis=1)
  return hk.Linear(1)(x)

gat_layer = GAT(
    attention_query_fn=lambda n: hk.Linear(8)(n),  # Applies W to the node features
    attention_logit_fn=attention_logit_fn,
    node_update_fn=None,
    add_self_edges=True,
)

In [20]:
def gat_definition(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
  """Defines a GAT network for the karate club node classification task.
  Args:
    graph: GraphsTuple the network processes.

  Returns:
    output graph with updated node values.
  """
  def _attention_logit_fn( sender_attr, receiver_attr, edges):
    del edges
    x = jnp.concatenate((sender_attr, receiver_attr), axis=1)
    return hk.Linear(1)(x)

  gn = GAT(
    attention_query_fn=lambda n: hk.Linear(64)(n),
    attention_logit_fn=_attention_logit_fn,
    node_update_fn=None,
    add_self_edges=True)
  graph = gn(graph)

  gn = GAT(
    attention_query_fn=lambda n: hk.Linear(32)(n),
    attention_logit_fn=_attention_logit_fn,
    node_update_fn=hk.Linear(42),
    add_self_edges=True)
  graph = gn(graph)
  return graph

In [22]:
network = hk.without_apply_rng(hk.transform(gat_definition))
result = optimize_email_dataset(network, num_steps=500)
# accuracy : %82.4 for 500 samples over 500 epochs
# accuracy : %93.3 for 800 samples over 500 epochs

step 0 accuracy 0.0060
step 1 accuracy 0.0846
step 2 accuracy 0.0945
step 3 accuracy 0.1055
step 4 accuracy 0.1015
step 5 accuracy 0.0915
step 6 accuracy 0.0796
step 7 accuracy 0.0736
step 8 accuracy 0.0896
step 9 accuracy 0.1522
step 10 accuracy 0.1711
step 11 accuracy 0.2010
step 12 accuracy 0.2269
step 13 accuracy 0.3045
step 14 accuracy 0.3383
step 15 accuracy 0.3552
step 16 accuracy 0.3682
step 17 accuracy 0.3801
step 18 accuracy 0.3990
step 19 accuracy 0.4169
step 20 accuracy 0.4448
step 21 accuracy 0.4716
step 22 accuracy 0.5005
step 23 accuracy 0.5323
step 24 accuracy 0.5642
step 25 accuracy 0.5871
step 26 accuracy 0.6239
step 27 accuracy 0.6418
step 28 accuracy 0.6547
step 29 accuracy 0.6597
step 30 accuracy 0.6756
step 31 accuracy 0.6816
step 32 accuracy 0.6915
step 33 accuracy 0.7124
step 34 accuracy 0.7323
step 35 accuracy 0.7512
step 36 accuracy 0.7662
step 37 accuracy 0.7791
step 38 accuracy 0.7960
step 39 accuracy 0.8060
step 40 accuracy 0.8229
step 41 accuracy 0.8368
st

In [334]:
########## Harmonic Function ##########

# Still under construction. Previously got the accuracy as %100 for Karate Dataset. Now trying to implement it for email dataset.

def email_graph():
    G = nx.Graph()
    len(edges)
    for i in range(len(edges)):
      G.add_edge(edges[i][0],edges[i][1])
    return G

In [335]:
G = email_graph()

In [336]:
def predicted(G):
  split = np.arange(1005)
  np.random.shuffle(split)
  for i in split[:800]:
    G.nodes[i]["label"] = f'{nodes_and_labels[i][1]}'
  predicted = nx.node_classification.harmonic_function(G, max_iter=500)
  print(predicted)
  return predicted

predicted(G)
print(G.nodes)

def resulted(G):

    node_array = np.array(G.nodes)
    prediction = np.array(predicted(G))
    resulted = [None] * len(node_array)
    resulted = np.array(resulted)

    for i in range(len(node_array)):
        resulted[node_array[i]] = prediction[i]

    return resulted

['1', '1', '21', '21', '21', '25', '21', '14', '14', '14', '9', '14', '14', '26', '4', '17', '34', '1', '1', '14', '9', '9', '9', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '4', '34', '14', '14', '17', '17', '10', '10', '36', '37', '5', '7', '4', '22', '21', '21', '21', '21', '21', '7', '7', '36', '21', '25', '4', '8', '15', '15', '15', '37', '37', '9', '1', '1', '10', '10', '3', '36', '3', '29', '15', '4', '36', '37', '1', '36', '34', '20', '20', '8', '8', '9', '4', '5', '4', '10', '16', '16', '16', '16', '16', '38', '7', '7', '36', '38', '36', '8', '27', '8', '8', '8', '10', '10', '13', '13', '36', '26', '10', '1', '36', '0', '13', '16', '16', '22', '6', '5', '4', '0', '28', '28', '4', '2', '13', '13', '21', '21', '17', '17', '14', '36', '8', '40', '35', '15', '23', '0', '0', '7', '10', '36', '27', '35', '1', '0', '7', '19', '19', '36', '14', '37', '24', '17', '13', '36', '4', '4', '13', '13', '10', '4', '38', '32', '32

In [337]:
real = np.asarray(nodes_and_labels)
real = real[:,1]
real = list(map(str,real))

In [338]:
data = np.array(real)

predicted = resulted(G)
print(predicted)

['1', '1', '21', '21', '21', '25', '21', '14', '14', '14', '9', '14', '14', '26', '4', '17', '34', '1', '1', '14', '9', '9', '9', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '10', '5', '34', '14', '14', '17', '17', '10', '10', '36', '37', '5', '7', '4', '22', '22', '21', '21', '21', '21', '7', '7', '36', '21', '25', '4', '8', '15', '15', '15', '37', '37', '9', '1', '1', '10', '10', '3', '3', '3', '29', '15', '36', '36', '37', '1', '36', '34', '20', '20', '8', '15', '9', '4', '5', '4', '10', '16', '16', '16', '16', '16', '38', '7', '7', '15', '38', '36', '8', '27', '8', '8', '8', '10', '10', '13', '13', '6', '26', '10', '1', '36', '0', '13', '16', '16', '22', '6', '5', '4', '0', '28', '28', '4', '2', '13', '13', '21', '21', '17', '17', '14', '36', '8', '40', '35', '15', '23', '0', '0', '7', '10', '36', '27', '35', '35', '0', '7', '19', '19', '36', '14', '37', '24', '17', '13', '36', '4', '4', '13', '13', '10', '4', '38', '32', '3

In [339]:
correct = 0
for i in range(len(real)):
        if real[i] == predicted[i]:
                correct += 1
miss = len(real)-correct
accuracy = correct/len(real)
print("Accuracy is %f " %accuracy)
print("Hits are %d and misses are %d" %(correct,miss))

# Note that for each experiment, I have recorded 5 accuracies that I observed as the output.

# Harmonic function max_iteration = 3    =>   For the train-split rate of %50, the accuracy is %93.23 - %91.84 - %92.94 - %93.54 - %92.84
# Harmonic function max_iteration = 10   =>   For the train-split rate of %50, the accuracy is %91.34 - %93.03 - %92.14 - %92.73 - %91.84
# Harmonic function max_iteration = 500   =>   For the train-split rate of %50, the accuracy is %92.64 - %92.94 - %93.23 - %93.43 - %92.34

# Harmonic function max_iteration = 3    =>   For the train-split rate of %80, the accuracy is %99.3 - %98.91 - %98.91 - %98.41 - %99.10
# Harmonic function max_iteration = 10   =>   For the train-split rate of %80, the accuracy is %99.0 - %98.51 - %98.61 - %99.01 - %98.41
# Harmonic function max_iteration = 500   =>   For the train-split rate of %80, the accuracy is %98.81 - %98.71 - %98.51 - %98.81 - %97.51

Accuracy is 0.975124 
Hits are 980 and misses are 25
